In [ ]:
# Load the Data
%run data_prep.ipynb

In [ ]:
# EDA
'''
- 
- 
- Visualize the sampling per neighborhood
- Distribution of the restaurant tags
- Distribution of the has images, phone, and address
- Distribution of the prices
- Histogram of the calculated score
'''
import matplotlib.pyplot as plt
%matplotlib inline


# Will need to make these graphs more presentable

# Distribution of review count
plt.hist(bus_copy['review_count'], color='blue', edgecolor='black')
plt.axvline(bus_copy['review_count'].median(), color='k', linestyle='dashed', linewidth=1)
plt.xlabel('Review Count')
plt.ylabel('No. of Restaurants')
plt.title('Log Distribution of Review Counts')
plt.yscale('log')
plt.show()

# Distribution of balanced rating score
plt.hist(bus_copy['brs'], color='blue', edgecolor='black')
plt.axvline(bus_copy['brs'].median(), color='k', linestyle='dashed', linewidth=1)
plt.xlabel('BRS')
plt.ylabel('No. of Restaurants')
plt.title('Distribution of BRS')
plt.show()

# Distribution of rating
rating_counts = bus_copy.loc[:, '0.0':'5.0'].sum()
rating_counts.plot(kind='bar', color='skyblue')
plt.title('Rating Counts')
plt.xlabel('Ratings')
plt.ylabel('Count')
plt.xticks(rotation=90)  # Rotate x-axis labels if needed
plt.show()

# Count of each neighborhood
neighborhood_counts = bus_copy.loc[:, 'Allston':'West Roxbury'].sum()
neighborhood_counts.plot(kind='bar', color='skyblue')
plt.title('Neighborhood Counts')
plt.xlabel('Neighborhoods')
plt.ylabel('Count')
plt.xticks(rotation=90)  # Rotate x-axis labels if needed
plt.show()


In [ ]:
# Counts of prices
price_counts = bus_copy.loc[:, ['affordable_eats', 'mid-range_dining', 'upscale_dining', 'gourmet_experience']].sum()

print(price_counts)

price_counts.plot(kind='bar', color='skyblue')
plt.title('Price Counts')
plt.xlabel('Prices')
plt.ylabel('Count')
plt.xticks(rotation=50)  # Rotate x-axis labels if needed
plt.show()


# Counts of tags
tag_counts = bus_copy.loc[:, ['delivery', 'pickup', 'restaurant_reservation', 'has_image', 'has_phone', 'has_st_add', 'has_price']].sum()

print(tag_counts)

tag_counts.plot(kind='bar', color='skyblue')
plt.axhline(y = bus_copy.shape[0], color = 'r', linestyle = '-')
plt.title('Tag Counts')
plt.xlabel('Tags')
plt.ylabel('Count')
plt.xticks(rotation=50)  # Rotate x-axis labels if needed
plt.show()





In [ ]:
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
from shapely.geometry import Point, Polygon
%matplotlib inline

bos_map = gpd.read_file('City_of_Boston_Boundary/City_of_Boston_Boundary.shp')

geometry = [Point(xy) for xy in zip(bus_copy['longitude'], bus_copy['latitude'])]
crs = {'init':'epsg:4326'}

geo_df = gpd.GeoDataFrame(bus_copy, # specify our data
                          crs=crs, # specify our coordinate reference system
                          geometry=geometry) # specify the geometry list we created

fig, ax = plt.subplots(figsize=(15,15))
bos_map.plot(ax=ax, alpha=0.4, color='grey')

geo_df.plot(ax=ax,
            markersize=2, 
            color='blue', 
            marker='o')


plt.legend(prop={'size':15})

In [ ]:
import folium
geo_df.explore(
    column="neighborhood",  
    tooltip=["name", 'neighborhood', 'id', 'distance', 'zip_code'],
    popup=True,  # show all values in popup (on click)
    tiles="CartoDB positron",  # use "CartoDB positron" tiles
    cmap="Set1",  # use "Set1" matplotlib colormap
)

In [ ]:
#stats = bus_copy.describe().transpose()
bos_map = gpd.read_file('Boston_Neighborhoods/Boston_Neighborhoods.shp')
ax = geoplot.polyplot(bos_map, projection=gcrs.AlbersEqualArea(), zorder=2, figsize=(15, 15))
geoplot.kdeplot(geo_df, cmap='Reds', thresh=0, fill=True, clip=bos_map, ax=ax)


In [ ]:
print(bus_copy['brs'].quantile(0.75))
print(bus_copy['brs'].median())

cuisines = bus_copy.loc[:, 'acaibowls':'wraps'].columns

In [ ]:
cuisine_counts = (bus_copy.loc[:, 'acaibowls':'wraps'].sum().to_frame()).sort_values([0], ascending = False)

cuisine_counts[:30].plot(kind='bar', color='skyblue')
plt.title('Cuisine Counts')
plt.xlabel('Cuisines')
plt.ylabel('Count')
plt.xticks(rotation=90)  # Rotate x-axis labels if needed
plt.show()

# explore the distribution of brs in each feature
# what neighborhoods have the highest mean brs
# next up is machine learning!

In [ ]:
bus_copy.groupby(["neighborhood"])["brs"].median().to_frame().plot(kind='bar', color='skyblue')
plt.axhline(y = bus_copy['brs'].median(), color = 'r', linestyle = '-')
plt.title('Median BRS By Neighborhood')
plt.xlabel('Neighborhood')
plt.ylabel('Median')
plt.xticks(rotation=90)  # Rotate x-axis labels if needed
plt.show()



In [ ]:
bus_copy.boxplot(column='brs',by='high_brs')
bus_copy.boxplot(column='review_count',by='high_brs')
bus_copy.groupby(['neighborhood', 'high_brs'])['id'].count().unstack('high_brs').plot.bar()

